# Sesión 4 · Libro de trabajo (ejercicios)
**Visualización como criterio, no como exploración**

Este notebook está diseñado para practicar el *juicio previo al modelado* **sin entrenar modelos** ni optimizar métricas.

## Reglas de trabajo
- En el **Ejercicio 1 (Diagnóstico ciego)** se evalúa tu **diagnóstico argumentado**, no “adivinar el generador”.
- Herramientas permitidas en el diagnóstico ciego:
  - `plt.scatter` / `sns.scatterplot`
  - `sns.histplot`
  - (opcional) `sns.boxplot` para outliers

> Los datasets del diagnóstico ciego se cargan desde un fichero `.npz` para evitar spoilers por fórmulas.


In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

sns.set_theme()


## 0. Carga de datasets (NO editar)

Cargamos `A`, `B`, `C` desde `datasets_sesion4.npz`.

- `A` y `B` tienen una sola variable `X` y un objetivo `y`.
- `C` tiene dos variables en `X` (con escalas muy distintas) y un objetivo `y`.


In [ ]:
data = np.load("datasets_sesion4.npz", allow_pickle=True)

A = data["A"].item()
B = data["B"].item()
C = data["C"].item()
D = data["D"].item()

def get_xy(dataset):
    X = dataset["X"]
    y = dataset["y"]
    if X.ndim == 1:
        X = X.reshape(-1, 1)
    return X, y

print("Datasets cargados:", list(data.files))
print("A shapes:", get_xy(A)[0].shape, get_xy(A)[1].shape)
print("B shapes:", get_xy(B)[0].shape, get_xy(B)[1].shape)
print("C shapes:", get_xy(C)[0].shape, get_xy(C)[1].shape)
print("D shapes:", get_xy(D)[0].shape, get_xy(D)[1].shape)


---
# 🟢 Ejercicio 1 · Diagnóstico ciego (A, B, C)

Para cada dataset:
1) Visualiza con scatter.
2) Observa distribución/outliers con histograma y/o boxplot.
3) Rellena la ficha de diagnóstico (Markdown) y justifica.

### Ficha de diagnóstico
- **Relación**: lineal / no lineal / sin señal clara
- **Ruido**: bajo / medio / alto
- **Outliers**: no / sí / sospecha
- **Escala** (si aplica): equilibrada / dispareja
- **Expectativa razonable**: ¿qué esperas que ocurra con un baseline simple en Sesión 5?
- **Hipótesis de baseline** (sin entrenar): lineal / árboles / distancia (KNN) — justifica


In [ ]:
def plot_1d_dataset(name, dataset):
    X, y = get_xy(dataset)
    x = X[:, 0]

    fig, ax = plt.subplots(figsize=(6, 4))
    ax.scatter(x, y, s=18)
    ax.set_title(f"{name}: scatter (X vs y)")
    ax.set_xlabel("X")
    ax.set_ylabel("y")
    plt.show()

    fig, ax = plt.subplots(figsize=(6, 4))
    sns.histplot(y, bins=30, ax=ax)
    ax.set_title(f"{name}: hist(y)")
    plt.show()

    fig, ax = plt.subplots(figsize=(4, 5))
    sns.boxplot(y=y, ax=ax)
    ax.set_title(f"{name}: boxplot(y)")
    plt.show()


In [ ]:
def plot_2d_classification(name, dataset):
    X, y = get_xy(dataset)
    if X.shape[1] != 2:
        raise ValueError("Este plot espera X con 2 features.")
    cols = dataset.get("feature_names", ["x1", "x2"])

    fig, ax = plt.subplots(figsize=(6, 5))
    sns.scatterplot(x=X[:, 0], y=X[:, 1], hue=y, palette="deep", s=28, ax=ax)
    ax.set_title(f"{name}: scatter (X1 vs X2) con clases")
    ax.set_xlabel(cols[0])
    ax.set_ylabel(cols[1])
    ax.legend(title="Clase")
    plt.show()


## Dataset A · Visualiza y diagnostica

In [ ]:
plot_1d_dataset("A", A)

**Tu diagnóstico (A):**
- Relación: Lineal
- Ruido: Bajo/medio
- Outliers: No
- Expectativa razonable: Que es muy probable que el modelo funcione de manera estable 
- Hipótesis de baseline y justificación (3–6 líneas): Modelo simple 


## Dataset B · Visualiza y diagnostica

In [ ]:
plot_1d_dataset("B", B)

**Tu diagnóstico (B):**
- Relación: Lineal
- Ruido: Bajo/medio
- Outliers: Si
- Expectativa razonable: El modelo estará sesgado
- Hipótesis de baseline y justificación (3–6 líneas): Seguramente haya fallos sistematicos por culpa de los outliers


## Dataset C · Visualiza y diagnostica (escalas dispares)

In [ ]:
Xc, yc = get_xy(C)
cols = C.get("feature_names", ["x1", "x2"])

fig, ax = plt.subplots(figsize=(6, 4))
ax.scatter(Xc[:, 0], yc, s=18)
ax.set_title("C: scatter (feature_small vs y)")
ax.set_xlabel(cols[0])
ax.set_ylabel("y")
plt.show()

fig, ax = plt.subplots(figsize=(6, 4))
ax.scatter(Xc[:, 1], yc, s=18)
ax.set_title("C: scatter (feature_large vs y)")
ax.set_xlabel(cols[1])
ax.set_ylabel("y")
plt.show()

fig, ax = plt.subplots(figsize=(6, 4))
sns.histplot(Xc[:, 0], bins=30, ax=ax)
ax.set_title(f"C: hist({cols[0]})")
plt.show()

fig, ax = plt.subplots(figsize=(6, 4))
sns.histplot(Xc[:, 1], bins=30, ax=ax)
ax.set_title(f"C: hist({cols[1]})")
plt.show()


**Tu diagnóstico (C):**
- ¿Qué variable “pesa” más numéricamente? feature_large
- ¿Qué riesgo aparece si usas modelos basados en distancia sin escalar? El modelo estará sesgado
- ¿Qué esperas de un baseline sin escalar? Las mediciones no se ajustarán, teniendo un sesgo numérico
- Hipótesis de baseline y justificación (3–6 líneas):


## Dataset D · Diagnóstico ciego (clasificación con solapamiento)

**Objetivo:** observar separabilidad y solapamiento **sin entrenar modelos**.

Pistas para tu diagnóstico:
- ¿Las clases parecen separables en este plano?
- ¿Ves solapamiento fuerte o moderado?
- Si el solapamiento es alto: ¿qué implica sobre el “techo” de rendimiento?


In [ ]:
plot_2d_classification("D", D)

**Tu diagnóstico (D):**
- Separabilidad visual: baja
- ¿Hay solapamiento? ¿Dónde? Si, en todos lados
- Expectativa razonable: ¿qué tipo de errores serían inevitables? Que da igual el modelo, que va a funcionar mal. Si el modelo funciona bien es porque tendrá superajuste. 
- Hipótesis de baseline (sin entrenar) y justificación (3–6 líneas): Usaremos uno simple para confirmar que no hay por donde coger estos datos. 


---
# 🟡 Ejercicio 2 · La trampa del “Orden Fantasma” (Encoding)

Dataset pequeño y visible. Objetivo: **entender por qué algunas codificaciones crean distancias falsas**.


In [ ]:
df_inmo = pd.DataFrame({
    "ID": range(1, 11),
    "Ciudad": ["Madrid", "Soria", "Madrid", "Madrid", "Soria", "Madrid", "Soria", "Soria", "Madrid", "Soria"],
    "Orientacion": ["Norte", "Sur", "Este", "Oeste", "Norte", "Sur", "Oeste", "Este", "Norte", "Sur"],
    "Calidad_Materiales": ["Alta", "Media", "Alta", "Baja", "Media", "Alta", "Baja", "Media", "Alta", "Baja"],
    "Precio": [320, 140, 310, 210, 150, 330, 190, 160, 325, 200],
})
df_inmo


### 2.1 “Label Encoding” manual para `Ciudad` (simulación)

Asigna números a las ciudades y calcula la “distancia” (resta) entre Madrid y Soria.


In [ ]:
city_map = {"Madrid": 1, "Soria": 2}
df_inmo["Ciudad_LE"] = df_inmo["Ciudad"].map(city_map)
df_inmo[["Ciudad", "Ciudad_LE"]].drop_duplicates()


In [ ]:
dist_mad_soria = city_map["Soria"] - city_map["Madrid"]
print("Distancia (Soria - Madrid) con Label Encoding:", dist_mad_soria)


> **⚠️ Cuidado con el “Orden Fantasma”:**  
> Si usas números para categorías sin orden, el modelo interpreta “distancias” que **no existen**.


### 2.2 One-Hot con pandas (sin sklearn)

Genera one-hot para `Orientacion` con `pd.get_dummies`.


In [ ]:
ohe = pd.get_dummies(df_inmo["Orientacion"], prefix="Ori")
pd.concat([df_inmo[["Orientacion"]], ohe], axis=1).head()


### Reflexión (4–8 líneas)

- ¿Por qué usar “label” en `Calidad_Materiales` puede ser razonable (ordinal)? Porque es una categoria ordenada, relacionada. 
- ¿Por qué usar “label” en `Ciudad` es un error (nominal)? Porque estaríamos relacionando Ciudades sin ninguna intención de relacionar


---
# 🔴 Ejercicio 3 · Detective de Data Leakage (caso real)

**Contexto:** predecir si un paciente será reingresado en < 30 días.

Variables: Edad, Tensión arterial, Medicación previa, Fecha de alta, Nombre/ID del médico que firmó el reingreso.

### 3.1 Juicio (sin código)
1) ¿Qué variable sospechas que provoca *data leakage* masivo? Nombre/ID del médico que firmó el reingreso
2) Explica por qué (disponibilidad temporal + proceso). 


### 3.2 Mini-experimento (limitado): correlación extrema

Creamos un dataset de juguete con una columna “chivato” casi idéntica a `y`.


In [ ]:
rng = np.random.default_rng(7)
m = 300
y = rng.integers(0, 2, size=m)

edad = rng.normal(55, 12, size=m)
tension = rng.normal(125, 18, size=m)

# "chivato": casi igual a y (leak)
chivato = y + rng.normal(0, 0.01, size=m)

df_leak = pd.DataFrame({
    "edad": edad,
    "tension": tension,
    "chivato_del_futuro": chivato,
    "y": y
})

corr = df_leak.corr(numeric_only=True)["y"].sort_values(ascending=False)
corr


In [ ]:
# Regla automática (demo): eliminar columnas con correlación > 0.99 con y
target = "y"
threshold = 0.99

corr_to_y = df_leak.corr(numeric_only=True)[target].abs()
to_drop = corr_to_y[corr_to_y > threshold].index.drop(target)
print("Columnas a eliminar por regla automática:", list(to_drop))


### Conclusión obligatoria (6–10 líneas)

- ¿Es suficiente esta automatización para detectar leakage? En este caso si
- Da **dos ejemplos** de leakage real que NO dependan de correlación extrema. 


---
# 🏁 Reto final · “Contrato de preprocesamiento” (puente a Sesión 5)

Escribe tu “plan de ataque” para Sesión 5.

1) Diagnóstico principal  
2) Riesgo principal  
3) Decisión de escalado  
4) Decisión de encoding  
5) Leaky feature vigilada  
6) Baseline razonable a probar primero


*(Escribe aquí tu contrato)*